In [1]:
import re
import glob
import signal
import pandas as pd
import numpy as np
from random import sample
from datetime import datetime 
from itertools import product
from SetSimilaritySearch import all_pairs
# for CV
from sklearn.model_selection import GroupKFold
# process mining
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
# miners
from pm4py.algo.discovery.inductive import algorithm as inductive_miner
from pm4py.algo.discovery.heuristics import algorithm as heuristics_miner
# performance metrics
from pm4py.algo.evaluation.replay_fitness import algorithm as replay_fitness_evaluator
from pm4py.algo.evaluation.precision import algorithm as precision_evaluator
from pm4py.algo.evaluation.generalization import algorithm as generalization_evaluator
from pm4py.algo.evaluation.simplicity import algorithm as simplicity_evaluator

In [2]:
def similarity_check(d=None, key_indices=None, n_samples=10, min_cosine_sim=0.5):
    event_log_df = d.iloc[:, key_indices]
    event_log_df.columns.values[0] = 'case:concept:name'
    event_log_df.columns.values[1] = 'time:timestamp'
    event_log_df.columns.values[2] = 'concept:name'
    # get the number of cases
    n_cases = len(set(event_log_df['case:concept:name']))
    if n_cases > n_samples:
        # sample n_sample cases
        sampled_case_ids = sample(list(set(event_log_df['case:concept:name'])), n_samples)
        event_log_df = event_log_df[event_log_df['case:concept:name'].isin(sampled_case_ids)]
    
    # get the sequence of activities by case_ids
    sets = event_log_df.groupby('case:concept:name')['concept:name']\
        .apply(list).apply(set).tolist()
    if len(sets) > 0:
        pairs = list(all_pairs(sets, similarity_func_name="cosine", similarity_threshold=0.1))
        #  print('pairs:', pairs)
        if len(pairs) == 0:
            cosine_sim = 0
        else:
            cosine_sim = np.mean([list(pairs[i])[2] for i in range(len(pairs))])
    else:
        cosine_sim = 0

    # print('cosine similarity:', cosine_sim)
    if cosine_sim <= min_cosine_sim:
        return False
    else:
        # if passed all preliminary tests, then return True
        return True

In [3]:
def measure_score(d=None, key_indices=None, miner='inductive_miner', metric='Buijs2014', n_splits=2, \
                  model_discovery_timeout_in_sec=5, evaluation_timeout_in_sec=30):
    def discover_model(event_log, miner):
        # default parameters are used
        if miner == 'inductive_miner':
            net, im, fm = inductive_miner.apply(event_log,
                {pm4py.algo.discovery.inductive.variants.im.algorithm.Parameters.NOISE_THRESHOLD: 0.2},
                pm4py.algo.discovery.inductive.algorithm.Variants.IM)
        elif miner == 'heuristics_miner':
            net, im, fm = heuristics_miner.apply(event_log, {
                heuristics_miner.Variants.CLASSIC.value.Parameters.DEPENDENCY_THRESH: 0.5,
                heuristics_miner.Variants.CLASSIC.value.Parameters.AND_MEASURE_THRESH: 0.65,
                heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_ACT_COUNT: 1,
                heuristics_miner.Variants.CLASSIC.value.Parameters.MIN_DFG_OCCURRENCES: 1,
                heuristics_miner.Variants.CLASSIC.value.Parameters.DFG_PRE_CLEANING_NOISE_THRESH: 0.05,
                heuristics_miner.Variants.CLASSIC.value.Parameters.LOOP_LENGTH_TWO_THRESH: 2})
        return net, im, fm

    def evaluate_score(event_log, net, im, fm, metric):
        if metric == 'fitness':
            score = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    variant=replay_fitness_evaluator.Variants.TOKEN_BASED)['log_fitness']
        elif metric == 'precision':
            score = precision_evaluator.apply(event_log, net, im, fm, 
                    variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
        elif metric == 'generalization':
            score = generalization_evaluator.apply(event_log, net, im, fm)
        elif metric == 'simplicity':
            score = simplicity_evaluator.apply(net)
        else:
            if metric == 'Buijs2014':
            # A paper "Quality dimensions in process discovery: The importance of fitness, 
            # precision, generalization and simplicity" proposed to calculate the following 
            # four metrics with giving 10 times more weight to replay fitness than 
            # the other three.
            # 10 x + 3x = 1 => x = 1 / 13
                weights = [10/13, 1/13, 1/13, 1/13]
            elif metric == 'average':
                weights = [0.25, 0.25, 0.25, 0.25]
            fitness = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    variant=replay_fitness_evaluator.Variants.TOKEN_BASED)['log_fitness']
            precision = precision_evaluator.apply(event_log, net, im, fm, 
                    variant=precision_evaluator.Variants.ETCONFORMANCE_TOKEN)
            #  fitness = replay_fitness_evaluator.apply(event_log, net, im, fm, 
                    #  variant=replay_fitness_evaluator.Variants.ALIGNMENT_BASED)['log_fitness']
            #  precision = precision_evaluator.apply(event_log, net, im, fm, 
                    #  variant=precision_evaluator.Variants.ALIGN_ETCONFORMANCE)
            generalization = generalization_evaluator.apply(event_log, net, im, fm)
            simplicity = simplicity_evaluator.apply(net)
            score = np.dot(weights, [fitness, precision, generalization, simplicity])
        return score
    
    def timeout_handler(signum, frame):
        raise Exception('timeout')
        
    def cross_validation(event_log_df, train, test):
        # dataset need be sorted by timestamp (https://pm4py.fit.fraunhofer.de/documentation#item-import-csv)
        #  print("%s %s" % (train, test))
        train_log_df = event_log_df.iloc[train]
        train_log_df = train_log_df.sort_values('time:timestamp')
        #  print('train_log_df')
        #  print(set(train_log_df['case:concept:name']))
        test_log_df = event_log_df.iloc[test]
        test_log_df = test_log_df.sort_values('time:timestamp')
        #  print('test_log_df')
        #  print(set(test_log_df['case:concept:name']))
        train_log = log_converter.apply(train_log_df, \
                                        variant=log_converter.Variants.TO_EVENT_LOG)
        test_log = log_converter.apply(test_log_df, \
                                       variant=log_converter.Variants.TO_EVENT_LOG)
    
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), \
              'discovering a process model...')
        #  print(train_log_df[['case:concept:name', 'time:timestamp', 'concept:name']])
        #  print(train_log_df.groupby('case:concept:name').size())
        #  print(set(train_log_df['case:concept:name']))
        #  print(set(train_log_df['time:timestamp']))
        #  print(set(train_log_df['concept:name']))
        
        # discover_model may take time. hence, we set a timeout for it
        # https://stackoverflow.com/a/494273/7184459
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(model_discovery_timeout_in_sec)
        try:
            net, im, fm = discover_model(train_log, miner)
        except Exception:
            print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'gave up discovering a model')
            print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'score evaluated:', 0)
            return 0
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'process model discovered')
        # reset a timer when a model discovery finishes before the deadline
        signal.alarm(0)
        # set another timer for evaluation
        signal.signal(signal.SIGALRM, timeout_handler)
        signal.alarm(evaluation_timeout_in_sec)
        try:
            score = evaluate_score(test_log, net, im, fm, metric)
        except Exception:
            print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'gave up evaluating a model')
            print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'score evaluated:', 0)
            return 0
        signal.alarm(0)
        print(datetime.now().strftime('%Y-%m-%d %H:%M:%S'), 'score evaluated:', score)
        return score
    
    # assume the columns as case_id, timestamp, and activity from left
    event_log_df = d.iloc[:, key_indices]
    event_log_df.columns.values[0] = 'case:concept:name'
    event_log_df.columns.values[1] = 'time:timestamp'
    event_log_df.columns.values[2] = 'concept:name'
    # evaluate the goodness of such an assumption with CV
    gkf = GroupKFold(n_splits=n_splits)
    X = list(event_log_df.index)
    groups = list(event_log_df['case:concept:name'])
    scores = [cross_validation(event_log_df, train, test) \
          for train, test in gkf.split(X, groups=groups)]
    score = round(np.mean(scores), ndigits=3)
    print('final score:', score)
    return score

In [4]:
def get_combinations_to_try(dataset=None, thr_timestamp_uniqueness=0.5, \
                            thr_case_id_uniqueness=0.15, \
                            thr_activity_uniquness=0.35, verbose=False):
    # we want to narrow down the candidates for key columns as much as possible
    # 1. timestamp
    # a timestamp column should contain a digit in its values
    # hence we identify the columns that involve a digit
    tmp = dataset.apply(lambda x: all([bool(re.search(r'\d', str(value))) for value in set(x)]) and \
                        not any(x == ''), \
                        axis=0).tolist()
    # get column indices as timestamp candidates
    tmp = np.where(tmp)[0].tolist()
    # most of values in timestamp column should be unique
    tmp = dataset.iloc[:, tmp] \
        .apply(lambda x: len(set(x)) / len(x) < thr_timestamp_uniqueness, axis=0)
    timestamp_cand = np.where(tmp)[0].tolist()
    if verbose:
        print('Timestamp condidate indices:', timestamp_cand)
    # 2. case_id
    # case_id also should contain a digit (we leverage the timestamp_cand)
    # in addition, the same case_id must appear multiple times as there cannot be only one activty per case
    tmp = dataset.iloc[:, timestamp_cand] \
        .apply(lambda x: len(set(x)) / len(x) < thr_case_id_uniqueness, axis=0)
    # get the indices of them
    case_id_cand = np.array(timestamp_cand)[np.where(tmp)[0]].tolist()
    if verbose:
        print('Case_id condidate indices:', case_id_cand)
    # 3. activity
    # likewise, the same activity must appear multiple times and none of the values can be missing
    tmp = dataset.apply(lambda x: len(set(x)) / len(x) < thr_activity_uniquness and not any(x == ''), axis=0)
    activity_cand = np.where(tmp)[0].tolist()
    if verbose:
        print('Activity condidate indices:', activity_cand)
        
    # get every possible key columns' indices
    # exclude any duplicates as we cannot select the same column for key indices (e.g. case_id and activity, etc.)
    key_index_cand = [list(t) for t in product(case_id_cand, timestamp_cand, activity_cand) if len(set(t)) >= 3]
    # as similar activity sequences have to be observed by multiple cases (otherwise we cannot identify a process flow)
    # hence we rule out the cases where each sequence of activities do not share a common pattern
    # in this regrad, we use cosine similarity to measure the overlaps between activities
    tmp = [similarity_check(dataset, key_indices, n_samples=10, min_cosine_sim=0.5) \
        for key_indices in key_index_cand]
    combinations_to_try = np.array(key_index_cand)[np.array(tmp)]
    # check how many candidates we try
    if verbose:
        print('#candidates:', len(combinations_to_try))
    return combinations_to_try

In [5]:
def narrow_down_cands_and_measure_scores(dataset=None, verbose=False, \
                                         miner='inductive_miner', metric='Buijs2014', n_splits=3):
    cands = get_combinations_to_try(dataset=dataset, verbose=verbose)
    # after pre-filtering the candidates, try every combination to process discovery
    # the intuition behind it is that the correct key column combination would result in
    # a good score (e.g. fitness, precision, generalizability, simplicity)
    scores = [{'indices': key_indices, \
               'score': measure_score(dataset, key_indices, miner=miner, \
                                      metric=metric, n_splits=n_splits)} for key_indices in cands]
    return scores

In [6]:
def check_if_correct_labels_in(dataset=None, scores=None, combinations_to_try=None, top=None):
    # get the indicies of the key columns
    h = dataset.columns.values.tolist()
    correct_labels = [
        h.index('case:concept:name'),
        h.index('time:timestamp'),
        h.index('concept:name')
    ]
    # check if it is in the top k
    top_cand_indices = sorted(range(len(scores)), key=lambda i: scores[i])[-top:]
    return correct_labels in np.array(combinations_to_try)[np.array(top_cand_indices)].tolist()

In [7]:
dataset_names = glob.glob('../datasets/*.csv', recursive=False)
n_rows = 1000
datasets = [pd.read_csv(d, nrows=n_rows) for d in dataset_names]

In [ ]:
results = [narrow_down_cands_and_measure_scores(dataset=dataset, verbose=True, \
                                                miner='inductive_miner', metric='Buijs2014', n_splits=3) \
           for dataset in datasets]

Timestamp condidate indices: [0, 1, 2, 3, 4, 6, 7, 8, 9]
Case_id condidate indices: [0, 1, 2, 3, 4, 6, 7, 9]
Activity condidate indices: [0, 1, 2, 3, 4, 5, 6, 7, 9, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 24]
#candidates: 854
2022-07-21 05:34:27 discovering a process model...
2022-07-21 05:34:32 gave up discovering a model
2022-07-21 05:34:32 score evaluated: 0
2022-07-21 05:34:32 discovering a process model...
2022-07-21 05:34:37 gave up discovering a model
2022-07-21 05:34:37 score evaluated: 0
2022-07-21 05:34:37 discovering a process model...
2022-07-21 05:34:42 gave up discovering a model
2022-07-21 05:34:42 score evaluated: 0
final score: 0.0
2022-07-21 05:34:42 discovering a process model...
2022-07-21 05:34:47 gave up discovering a model
2022-07-21 05:34:47 score evaluated: 0
2022-07-21 05:34:47 discovering a process model...
2022-07-21 05:34:52 gave up discovering a model
2022-07-21 05:34:52 score evaluated: 0
2022-07-21 05:34:52 discovering a process model...
2022-07-21 0

replaying log with TBR, completed variants ::   0%|          | 0/441 [00:00<?, ?it/s]

2022-07-21 05:35:16 score evaluated: 0.8439685488151009
2022-07-21 05:35:16 discovering a process model...
2022-07-21 05:35:17 process model discovered


replaying log with TBR, completed variants ::   0%|          | 0/2 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/261 [00:00<?, ?it/s]

...

2022-07-21 10:51:27 score evaluated: 0.9833471800404645
2022-07-21 10:51:27 discovering a process model...
2022-07-21 10:51:27 process model discovered


replaying log with TBR, completed variants ::   0%|          | 0/8 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/142 [00:00<?, ?it/s]

replaying log with TBR, completed variants ::   0%|          | 0/8 [00:00<?, ?it/s]

2022-07-21 10:51:29 score evaluated: 0.9842812218606887
final score: 0.983
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
final score: 0.0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 10:51:29 gave up discovering a model
2022-07-21 10:51:29 score evaluated: 0
final score: 0.0
2022-07-21 10:51:29 discovering a process model...
2022-07-21 1

In [ ]:
# check if the correct tuple of key indices is in the top scores
scores = [result[cand]['score'] for cand in range(len(result))]
combinations_to_try = [result[cand]['indices'] for cand in range(len(result))]

is_in = [check_if_correct_labels_in(scores, correct_labels, combinations_to_try, top + 1) for top in range(10)]
{'dataset': dataset_names[1], 'is_in': is_in}